In [65]:
import numpy as np
import igl
import meshplot as mp
import math

In [5]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [382]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 8})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

In [538]:
def build_B(points):
    B = []
    is_empty =True
    
    for point in points:
        if is_empty:
            cur = []
            cur = np.hstack((cur,1))
            
            #x,y,z
            cur = np.hstack((cur,point[0]))
            cur = np.hstack((cur,point[1]))
            cur = np.hstack((cur,point[2]))

            #xy,xz,yz            
            cur = np.hstack((cur,point[0] * point[1]))
            cur = np.hstack((cur,point[1] * point[2]))
            cur = np.hstack((cur,point[1] * point[2]))

            #x^2, y^2, z^2
            cur = np.hstack((cur,point[0] * point[0]))
            cur = np.hstack((cur,point[1] * point[1]))
            cur = np.hstack((cur,point[2] * point[2]))
            is_empty = False
            B = np.hstack((B, cur))
        else:
            cur = []
            cur = np.hstack((cur,1))
            
            #x,y,z
            cur = np.hstack((cur,point[0]))
            cur = np.hstack((cur,point[1]))
            cur = np.hstack((cur,point[2]))

            #xy,xz,yz            
            cur = np.hstack((cur,point[0] * point[1]))
            cur = np.hstack((cur,point[1] * point[2]))
            cur = np.hstack((cur,point[1] * point[2]))

            #x^2, y^2, z^2
            cur = np.hstack((cur,point[0] * point[0]))
            cur = np.hstack((cur,point[1] * point[1]))
            cur = np.hstack((cur,point[2] * point[2]))
            B = np.vstack((B, cur))
    return B


    


def find_closed_point(point, points):
    cur_min = math.inf
    cur_min_point = []
    for p in points:
        if (np.linalg.norm(p - point) < cur_min and np.linalg.norm(p - point) > 0):
            cur_min = np.linalg.norm(p - point)
            cur_min_point = p
            
    return cur_min_point;
    
def add_point(points, p):
    if points.size == 0:
        points = np.append(points, p, axis=1)
    else:
        points = np.vstack((points, p))
    
    return points

def add_eps_point(points, eps, ni, size, constraint):
    for i in range(0, size):
        p = points[i] + eps * ni[i]
        
        
        points = add_point(points, p)
        constraint = np.append(constraint, eps)
        
        eps_alt = 0.5 * eps
        while(not np.array_equal(find_closed_point(points[i], points), p)):
            constraint = np.delete(constraint, constraint.size - 1, axis=0)
            points = np.delete(points, int(points.size / 3) - 1, axis=0)
            
            p = points[i] + eps_alt * ni[i]
            
            points = add_point(points, p)
            constraint = np.append(constraint, eps_alt)

            
            
            eps_alt = 0.5 * eps_alt
                    
                
    return points, constraint
    
def build_W(point, points):
    W = np.array([[]])
    for i in range(0, int(points.size / 3)):
        cur = []
        #print(cur)
        
        for j in range(0, i):
            cur.append(0)
        
        cur.append(w(np.linalg.norm([point - points[i]]), 30))
        #print(cur)
        
        for j in range(i + 1, int(points.size / 3)):
            cur.append(0)
            
        #print(cur)
        W = add_point(W, [cur])
        
    return W

def w(r, h):
    if(r < h):
        return pow(1 - r/h, 4) * (4 * r/h + 1)
    else:
        return 0
    

In [517]:
p = pi

min_index = np.argmin(p, axis = 0)
max_index = np.argmax(p, axis = 0)

min = np.array([p[min_index[0]][0], p[min_index[1]][1], p[min_index[2]][2]])
max = np.array([p[max_index[0]][0], p[max_index[1]][1], p[max_index[2]][2]])

bbox_diagonal = np.linalg.norm(max - min)

eps = 0.01 * bbox_diagonal

size = int(pi.size / 3)

constraint = np.array([])

for i in range(0, size):
    constraint = np.append(constraint, 0)



p, constraint = add_eps_point(p, eps, ni, size, constraint)


p, constraint = add_eps_point(p, -eps, ni, size, constraint)
    
#mp.plot(pi, shading={"point_size": 4})
#mp.plot(p, shading={"point_size": 4})


#print(build_W(p[0], p))

In [518]:
print(constraint)

[ 0.          0.          0.         ... -0.71617665 -0.71617665
 -0.71617665]


PART 2

In [519]:

a =np.array([1, 1, 1])
b = -a

color = np.array([[]])

for i in range(0, constraint.size):
    if (constraint[i] > 0):
        color = add_point(color, [[1, 0, 0]])
    elif (constraint[i] < 0):
        color = add_point(color, [[0, 1, 0]])
    else:
        color = add_point(color, [[0, 0, 1]])
    
print(color)
mp.plot(pi, shading={"point_size": 4})
mp.plot(p, c=color, shading={"point_size": 4})

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0120944…

In [520]:
x, T = tet_grid((20, 20, 20), min, max)


In [563]:
B = build_B(p)


print(p.size)

a = []

def build_d(xi, points, d):
    for p in points:
        d = np.append(d, np.linalg.norm(p - xi))
        
    return d
    
d = []

for xi in x:
    W = build_W(xi, p)
    
    Bt = np.transpose(B)
    
    BtW = np.matmul(Bt, W)
    left = np.matmul(BtW, B)
    
    d = build_d(xi, p, d)
    right = np.matmul(BtW, d)
    np.linalg.solve(left, right)
    
    break
    
print(d)

3294


LinAlgError: Singular matrix

In [560]:
print(B)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00 -1.00000000e-01 -6.00000000e+00 ...  1.00000000e-02
   3.60000000e+01  1.00000000e+00]
 [ 1.00000000e+00 -4.40000000e+00 -3.40000000e+00 ...  1.93600000e+01
   1.15600000e+01  1.00000000e+00]
 ...
 [ 1.00000000e+00  2.56869493e+01 -3.72164553e+01 ...  6.59819365e+02
   1.38506454e+03  9.24197980e+03]
 [ 1.00000000e+00  2.55009834e+01 -4.43315334e+01 ...  6.50300153e+02
   1.96528485e+03  9.20832360e+03]
 [ 1.00000000e+00  2.41842972e+01 -4.99002823e+01 ...  5.84880232e+02
   2.49003817e+03  9.25256726e+03]]


In [422]:
mp.plot(x, shading={"point_size": 1.5,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# MLS function

In [262]:
# Parameters
bbox_min = np.array([-1., -1., -1.])
bbox_max = np.array([1., 1., 1.])
bbox_diag = np.linalg.norm(bbox_max - bbox_min)

n = 10

In [430]:
# Generate grid n x n x n

x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

#Compute implicit sphere function
center = np.array([0., 0., 0.])
radius = 1
fx = np.linalg.norm(x-center, axis=1) - radius

#print(np.linalg.norm([-1.173, -1.173, -1.173]-center) - radius)
#print(center)
#print(radius)
#print(x)
#print(fx)

In [431]:
# Treshold fx to visualize inside outside
ind = np.zeros_like(fx)
ind[fx >= 0] = 1

ind[fx < 0] = -1
mp.plot(x, c=ind, shading={"point_size": 0.1,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Marching to extract surface

In [265]:
# Marcing tet to extract surface

sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
mp.plot(sv, sf, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…